# **Google trends by Day**

In [11]:
from pytrends.request import TrendReq

# hl: https://www.andiamo.co.uk/resources/iso-language-codes/

# tz Colombia: 300
data_path = '../data/'

pytrends = TrendReq(hl='es-CO', tz=300)
#pytrends = TrendReq(hl='en-US', tz=360) 
#pytrends = TrendReq(hl='es-CO', tz=300, timeout=(10,25), proxies=['https://34.203.233.13:80',], retries=2, backoff_factor=0.1, requests_args={'verify':False})

In [8]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

def generate_date_ranges(start_date, end_date, months_interval):
    """
    Generate date ranges with a specified interval.

    Parameters:
    - start_date (datetime): The start date.
    - end_date (datetime): The end date.
    - months_interval (int): The interval in months.

    Returns:
    - list: A list of tuples representing start and end dates for each interval.
    """

    # Define the interval
    interval = relativedelta(months=months_interval)

    # Initialize the list to store dates
    dates = []

    # Iterate over the date range
    current_date = start_date
    while current_date <= end_date:
        # Calculate the end date by adding the specified months interval
        end_of_interval = current_date + interval

        # Adjust the end date to the last day of the previous month
        end_of_interval -= timedelta(days=1)

        # Append the result to the list
        dates.append(current_date.strftime("%Y-%m-%d") +" " +end_of_interval.strftime("%Y-%m-%d"))

        # Move to the next iteration
        current_date = end_of_interval + timedelta(days=1)

    return dates

# Example usage:
start_date_example = datetime(2020, 1, 12)
end_date_example = datetime(2023, 8, 31)
months_interval_example = 6

result = generate_date_ranges(start_date_example, end_date_example, months_interval_example)

In [9]:
result

['2020-01-12 2020-07-11',
 '2020-07-12 2021-01-11',
 '2021-01-12 2021-07-11',
 '2021-07-12 2022-01-11',
 '2022-01-12 2022-07-11',
 '2022-07-12 2023-01-11',
 '2023-01-12 2023-07-11',
 '2023-07-12 2024-01-11']

In [12]:
# Doc: https://pypi.org/project/pytrends/
# Git: https://github.com/GeneralMills/pytrends
# Cat: https://github.com/pat310/google-trends-api/wiki/Google-Trends-Categories

kw_list = ["Policia"]
pytrends.build_payload(kw_list, cat=0, timeframe='2020-01-12 2020-07-11', geo='CO', gprop='')

In [13]:
data = pytrends.interest_over_time()

In [16]:
data

""


In [44]:
data.to_csv(data_path+'data_3y.csv')